In [12]:
import pandas as pd

In [13]:
bookings = pd.read_csv('bookings.csv', encoding = 'windows-1251', sep=';')

In [15]:
# We are looking at the first 7 entries
bookings_head = bookings.head(7)
bookings_head

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [16]:
# Replace the spaces with underscores, and put them in lowercase
for column in bookings.columns:
    column_rename = column.replace(' ', '_').lower()
    bookings = bookings.rename(columns={f'{column}':f'{column_rename}'})


In [20]:
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [47]:
# Users of the countries have made the largest number of successful bookings in the top 5
bookings.query('is_canceled == 0') \
.groupby('country') \
.agg({'is_canceled':'count'}) \
.sort_values('is_canceled', ascending=False).head()

,is_canceled
country,
PRT,21071
GBR,9676
FRA,8481
ESP,6391
DEU,6069


In [32]:
# How many nights do City Hotel type hotels book on average
print(round(bookings.query('hotel == "City Hotel"') \
.agg({'stays_total_nights':'mean'}),2))

# How many nights do Resort Hotel type hotels book on average
print(round(bookings.query('hotel == "Resort Hotel"') \
.agg({'stays_total_nights':'mean'}),2))

stays_total_nights    2.98
dtype: float64
stays_total_nights    4.32
dtype: float64


In [33]:
# the type of room assigned to the client differs from the one originally booked due to overbooking, how many such observations?
bookings.query('assigned_room_type != reserved_room_type').agg({'reserved_room_type':'count'})

reserved_room_type    14917
dtype: int64

In [34]:
# Which month was the most frequently booked in 2016?
print(bookings.query("arrival_date_year == 2016").groupby('arrival_date_month').agg({'arrival_date_month':'count'}).idxmax())

# Which month was the most frequently booked in 2017?
print(bookings.query("arrival_date_year == 2017").groupby('arrival_date_month').agg({'arrival_date_month':'count'}).idxmax())

arrival_date_month    October
dtype: object
arrival_date_month    May
dtype: object


In [49]:
# for which month were City Hotel bookings cancelled most often in 2015? 2016? 2017?
bookings.query('hotel == "City Hotel" and is_canceled == 1').groupby('arrival_date_year')['arrival_date_month'].value_counts()

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

In [36]:
# # Will look at the numerical characteristics of three columns: adults, children and babies. Which one has the highest average value?
bookings.agg({'adults':'mean', 'children':'mean','babies':'mean'}).idxmax()

'adults'

In [37]:
# Create total_kids by combining the children and babies columns.
bookings['total_kids'] = bookings.children + bookings.babies

In [38]:
# Для отелей какого типа среднее значение переменной оказалось наибольшим?
round(bookings.groupby('hotel').agg({'total_kids': 'mean'}),2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


In [41]:
# Not all bookings were completed successfully, how many customers were lost in the process?
bookings['has_kids'] = bookings.total_kids > 0
bookings['has_kids'].value_counts()    

has_kids
False    110058
True       9332
Name: count, dtype: int64

In [50]:
# let's check which user group has a higher churn rate with or without children
print(round(bookings.query('has_kids == True and is_canceled == 1').shape[0] / bookings.query('has_kids == True').shape[0] * 100,2))
print(round(bookings.query('has_kids == False and is_canceled == 1').shape[0] / bookings.query('has_kids == False').shape[0] * 100,2))

34.92
37.22
